In [1]:
import pandas as pd
import re
import gensim # thư viện NLP
from pyvi import ViTokenizer, ViPosTagger # thư viện NLP tiếng Việt
from pandas import DataFrame
import re

In [2]:
xl = pd.ExcelFile('data.xlsx')
for name in xl.sheet_names:
    df = xl.parse(name)

In [3]:
df

,Rating,Content
0,5,1234570000000000120924793754905402008253644144...
1,5,"""Cuộc hành trình ngàn dặm nào cũng phải bắt đầ..."
2,5,*Thật sự rất đáng tiền khi mua cuốn sách này đ...
3,5,","
4,5,.
...,...,...
11264,1,yêu cầu shop xác nhận đơn hàng dùm lâu quá rồi
11265,1,"YÊU CẦU TIKI DÁN TEM HÀNG DỂ VỠ, mua bia về là..."
11266,1,yêu cầu tiki làm việc với bên vận chuyển giao ...
11267,1,yêu câu tiki xác nhận có đúng chính xác là ino...


In [4]:
from collections import Counter

In [26]:
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\\\n\(\)]'

def remove_special_characters(batch):
    batch = re.sub(chars_to_ignore_regex, '', batch)
    return batch

In [27]:
def remove_number_characters(batch):
    batch = re.sub("\d+",'', batch)
    return batch

In [28]:
def remove_any_special_left(batch):
    # Loại bỏ các kí tự đặc biệt
    batch = gensim.utils.simple_preprocess(batch)
    batch = ' '.join(batch)
    # Sau khi loại bỏ kí tự đặc biệt thì file sẽ bị mất nhận diện chuỗi -> thêm dấu nháy vào chuỗi
    return batch

In [29]:
df.dropna(inplace=True)

In [30]:
df['Content'] = df['Content'].astype('str')

In [31]:
df['Content'] = df['Content'].apply(remove_number_characters)
df['Content'] = df['Content'].apply(remove_special_characters)
df['Content'] = df['Content'].apply(remove_any_special_left)

In [14]:
df

,Rating,Content
0,5,1234570000000000120924793754905402008253644144...
1,5,"""Cuộc hành trình ngàn dặm nào cũng phải bắt đầ..."
2,5,*Thật sự rất đáng tiền khi mua cuốn sách này đ...
3,5,","
4,5,.
...,...,...
11264,1,yêu cầu shop xác nhận đơn hàng dùm lâu quá rồi
11265,1,"YÊU CẦU TIKI DÁN TEM HÀNG DỂ VỠ, mua bia về là..."
11266,1,yêu cầu tiki làm việc với bên vận chuyển giao ...
11267,1,yêu câu tiki xác nhận có đúng chính xác là ino...


In [35]:
!pip install vncorenlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 396.2 kB/s eta 0:00:00m eta 0:00:010:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-py3-none-any.whl size=2645952 sha256=a0fc2a32cecd75db64440b2e1586423b5ce344f853925f7baa012605fdddcf32
  Stored in directory: /home/dh2409/.cache/pip/wheels/3c/e9/86/706911c12e307aeb9a2702399f0dad38d36f1d6f9dde8af35e
Successfully built vncorenlp


In [36]:
from vncorenlp import VnCoreNLP

In [37]:
rdrsegmenter = VnCoreNLP("VnCoreNLP/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m') 

In [38]:
def segment_text(batch):
    batch = rdrsegmenter.tokenize(batch)
    if len(batch) == 0:
        return ""
    else:
        return " ".join(batch[0]).strip()

In [39]:
df['Content'] = df['Content'].apply(segment_text)

In [40]:
df

,Rating,Content
0,5,
1,5,cuộc hành_trình ngàn dặm nào cũng phải bắt_đầu...
2,5,thật_sự rất đáng tiền khi mua cuốn sách này để...
3,5,
4,5,
...,...,...
11264,1,yêu_cầu shop xác_nhận đơn hàng dùm lâu quá rồi
11265,1,yêu_cầu tiki dán tem hàng dể vỡ mua bia về làm...
11266,1,yêu_cầu tiki làm_việc với bên vận_chuyển giao ...
11267,1,yêu câu tiki xác_nhận có đúng chính_xác là ino...


In [41]:
import numpy as np
indices = np.where(df['Content'] == "")

In [42]:
indices

(array([    0,     3,     4,     5,     7,     9,    11,    16,    19,
           44,   572,  1578,  1739,  2590,  3425,  4406,  4407,  4556,
         4879,  4880,  4883,  4936,  5319,  6697,  6758,  7332,  7333,
         7741,  7742,  7743,  7744,  7745,  7746,  7747,  7750,  7751,
         7752,  7761,  8550,  8941,  8967,  9568,  9569,  9632, 10253]),)

In [43]:
df.drop(indices[0],axis=0,inplace=True)

In [47]:
df['label'] = df['Rating'].apply(lambda x : 1 if x > 3 else 0)

In [52]:
df.reset_index(inplace=True, drop=True)

In [53]:
df

,Rating,Content,label
0,5,cuộc hành_trình ngàn dặm nào cũng phải bắt_đầu...,1
1,5,thật_sự rất đáng tiền khi mua cuốn sách này để...,1
2,5,khá đẹp và dễ dùng,1
3,5,bút dùng tốt màu_sắc đẹp đáng giánên mua,1
4,5,shipper giao hơi muộn tí thôi nha,1
...,...,...,...
11219,1,yêu_cầu shop xác_nhận đơn hàng dùm lâu quá rồi,0
11220,1,yêu_cầu tiki dán tem hàng dể vỡ mua bia về làm...,0
11221,1,yêu_cầu tiki làm_việc với bên vận_chuyển giao ...,0
11222,1,yêu câu tiki xác_nhận có đúng chính_xác là ino...,0


In [54]:
train_dataset = df.sample(frac=0.7,random_state=42)
test_dataset = df.drop(train_dataset.index)

In [55]:
train_dataset

,Rating,Content,label
4024,5,mình mua của tiki sp này từ năm_ngoái sinh em ...,1
9810,3,hô sa phâ không co nguyên ve bi mo me xâ,0
4682,5,nồi xài ok lắm hầm thịt cực_kì nhanh nhừ kết_c...,1
8821,4,này thì cũng được bị cái thời_gian nấu hơi lâu...,1
1501,5,đóng_gói kĩ nhân_viên giao hàng nhanh nhiệt_tì...,1
...,...,...,...
6683,5,tai nghe chụp tai này đẹp kèm cáp chuyển_đổi â...,1
3871,5,mèo rất thích ăn đòi ăn suốt ngày,1
7310,5,trang bán hàng uy_tín nhất nếu thêm nhiều chươ...,1
3402,5,rất hài_lòng,1


In [57]:
train_dataset[['Content','label']].to_csv('train.csv',index=False)
test_dataset[['Content','label']].to_csv('test.csv',index=False)

In [58]:
from transformers import AutoModel, AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification,TrainingArguments, Trainer
from datasets import load_dataset

tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

def prepare_dataset(batch):
    batch_ = tokenizer(batch["Content"], padding=True, truncation=True)
    batch_['labels'] = batch['label']
    return batch_

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [60]:
dataset = load_dataset('csv',data_files={'train' : 'train.csv',
                                         'val' : 'test.csv'})

dataset_prepare = dataset.map(prepare_dataset, remove_columns=dataset.column_names["train"], batch_size=4, num_proc=8, batched=True) 

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

model = AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base", num_labels=2)

training_args = TrainingArguments(
    output_dir="./phoBert-sa",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_prepare['train'],
    eval_dataset=dataset_prepare['val'],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

Using custom data configuration default-efe6183dcc17a0c1


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

/home/dh2409/miniconda3/envs/pDL/lib/python3.8/site-packages/datasets/packaged_modules/csv/csv.py:154: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  csv_file_reader = pd.read_csv(file, iterator=True, dtype=dtype, **self.config.read_csv_kwargs)


Dataset csv downloaded and prepared to /home/dh2409/.cache/huggingface/datasets/csv/default-efe6183dcc17a0c1/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


/home/dh2409/miniconda3/envs/pDL/lib/python3.8/site-packages/datasets/packaged_modules/csv/csv.py:154: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  csv_file_reader = pd.read_csv(file, iterator=True, dtype=dtype, **self.config.read_csv_kwargs)


...

#5:   0%|          | 0/106 [00:00<?, ?ba/s]

#3:   0%|          | 0/106 [00:00<?, ?ba/s]

Downloading:   0%|          | 0.00/518M [00:00<?, ?B/s]

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['

RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 1.96 GiB total capacity; 887.27 MiB already allocated; 6.19 MiB free; 924.00 MiB reserved in total by PyTorch) (malloc at /pytorch/c10/cuda/CUDACachingAllocator.cpp:289)
frame #0: c10::Error::Error(c10::SourceLocation, std::string const&) + 0x46 (0x7f5298889536 in /home/dh2409/miniconda3/envs/pDL/lib/python3.8/site-packages/torch/lib/libc10.so)
frame #1: <unknown function> + 0x1cf1e (0x7f5298ad2f1e in /home/dh2409/miniconda3/envs/pDL/lib/python3.8/site-packages/torch/lib/libc10_cuda.so)
frame #2: <unknown function> + 0x1df9e (0x7f5298ad3f9e in /home/dh2409/miniconda3/envs/pDL/lib/python3.8/site-packages/torch/lib/libc10_cuda.so)
frame #3: THCStorage_resize + 0x96 (0x7f5249ab53d6 in /home/dh2409/miniconda3/envs/pDL/lib/python3.8/site-packages/torch/lib/libtorch_cuda.so)
frame #4: at::native::(anonymous namespace)::resize_cuda_(at::Tensor&, c10::ArrayRef<long>, c10::optional<c10::MemoryFormat>) + 0x9aa (0x7f524b2d8e4a 